[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/bamacgabhann/IEOS2023/main?labpath=ieos2023%2F4_Raster_data.ipynb)

# Raster Data

For Vector data, we saw how numpy or arrow arrays are used to store vector coordinates and attributes.

In Raster data, we're dealing with not with points or lines or polygons, but with pixels, each of which covers a particular area. We don't define the coordinates of each pixel individually, but rather define the coordinates of the overall set of pixels which make up an image. So, in that sense, raster data is a very different data format. 

However, raster pixels wouldn't be of interest to us unless they contain data. That data could be the elevation of the land surface, red/green/blue colour values for a photograph, infrared bands, radar reflectance, temperatures...really, any kind of data which can vary heterogenously over an area. That data is numbers - numbers which can be stored in arrays, like numpy or arrow arrays. 

Just as GeoPandas provides the functionality to work with vector data stored in arrays, _Rasterio_ and _GDAL_ provide the functionality to work with raster data stored in arrays. So while the data format is different, there are strong comparisons in how the data is managed.

There are other libraries which depend on other array formats, but rasterio is still the most feature-complete library that I'm aware of.

This Notebook exercise is minimally adapted from the Raster tutorial in *Michael Dorman's _<a href='https://geobgu.xyz/presentations/p_2023_ogh/02-raster.html'>Working with Spatial Data in Python</a>_*.

Let's open some data.

## 1. Reading data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
import rasterio.plot
import rasterio.mask

In [ ]:
f02 = '../sample_data/HLS.S30.T33UXU.2022200T095559.v2.0.B02.tiff'
b02 = rasterio.open(f02)
b02

This is tile ID T33UXU from 2022-07-19, from the 30m resolution Harmonized Landsat Sentinel-2 (HLS) Surface Reflecance product.

The 'r' refers to 'reading mode'; if we were creating the file, we'd use 'w' for 'writing mode'.

In [ ]:
b02.meta

So our image is a 16-bit tiff (refer back to Data Types if you're not sure what that means). Note the count is 1 - this is just a single band.

In [ ]:
r02 = b02.read()
r02

Here, we can see the actual data making up the image. 16-bits means the values can range from -32768 to 32767.

In [ ]:
type(r02)

And there you get comfirmation that, just like vector data with GeoPandas, this data for this raster is a numpy array.

In [ ]:
rasterio.plot.show(b02, cmap='Greys_r')

## 2. Writing data

This is just one band: we have 3 others. Let's put them together. To do this, we're going to create a new file with a count of 4 - but otherwise the metadata will be the same as for this single band. So we'll copy that metadata, and just update the count.

In [ ]:
meta = b02.meta
meta.update(count=4)
meta

Count updated. Now let's add the other bands, so we can add them all:

In [ ]:
f03 = '../sample_data/HLS.S30.T33UXU.2022200T095559.v2.0.B03.tiff'
f04 = '../sample_data/HLS.S30.T33UXU.2022200T095559.v2.0.B04.tiff'
f08 = '../sample_data/HLS.S30.T33UXU.2022200T095559.v2.0.B08.tiff'

In [ ]:
with rasterio.open('hls.tif', 'w', **meta) as hls:
    files = [f02, f03, f04, f08]
    for index, file in enumerate(files, start=1):
        b = rasterio.open(file)
        hls.write(b.read(1), index)
        b.close()


In [ ]:
hls = rasterio.open('hls.tif')
hls.meta

In [ ]:
r = hls.read()
r

## 3. Processing raster data

r is now an array with four bands. However, there's two problems. 

The first is that if we look again at the meta information, we'll see that there's a 'nodata' value of -9999. That's common practice - but numpy doesn't know that, so any calculations on the array will produce some wrong data unless we change it.

The second is that the data is reflectance, which should be between 0 (no reflectance) and 1 (total reflectance). But data in the file is being stored as 16-bit integers to save space - and that's been achieved by multiplying the real values by 1,000. We need to reverse that to use the data properly. That means first changing the data type to _float_.

Fortunately, the hard part is understanding this. Doing it is easy.

In [ ]:
hls.nodata

In [ ]:
r[r == hls.nodata]

this code shows the array r, limited to where r has the nodata value.

In [ ]:
r.dtype

That confirms the data type. We will change that first:

In [ ]:
r = r.astype(np.float_)
r

In [ ]:
r.dtype

Now our array is stored as 64-bit floating point numbers.

Here's a quirk of numpy; it can't deal with values which aren't numbers in arrays of integers. We actually saw this all the way back at the start of the data types notebook, when we made a pandas series with an NA, and it returned a dtype of object. However, numpy _can_ deal with NaN (not a number) values in float dtype arrays. 

_Side note: this is an example of why the arrow format is replacing numpy arrays - it has been designed to avoid the problems like this in numpy. GeoArrow is still in development, and it can't come fast enough for me._

In [ ]:
r[r == hls.nodata]

Still works, because the values haven't changed, just how they're stored. But now we can change this:

In [ ]:
r[r == hls.nodata] = np.nan
r[r == hls.nodata]

No more -999 values! Now we can rescale:

In [ ]:
r = r * 0.0001
r

So that's great, but if we look at all the values, we can see that the processing has introduced a couple of errors:

In [ ]:
np.nanmin(r), np.nanmax(r)

Shouldn't have values above 1 or below 0. Fortunately, again it's easy to cut those:

In [ ]:
r[r < 0] = 0.0
r[r > 1] = 1.0
np.nanmin(r), np.nanmax(r)

## 4. Working with bands

We can extract particular data from the multiband raster using _slicing_.

The array could be represented as ```r[bands, rows, columns]```. 

For each of bands, rows, and columns, we can specify a number or a range. We can specify a range by slicing using the ```:``` symbol, which in Python indicates all values. For example, ```3:``` means all values from 3 to the end, and ```:-1``` means all values except the last one. (Do remember - indices in Python start at 0, not 1; ```1:``` is not the whole list, ```0:``` is.)

Here's how we would specify the first 3 bands, including all rows and columns of those bands:

In [ ]:
r[:3, :, :]

In fact, we don't need the ```:``` where it's not actually specifying a subset, so we can just do

In [ ]:
r[:3]

This is bands 2, 3, and 4 - blue, green, and red. We can pass this array to matplotlib to plot it - but matplotlib is expecting the order R, G, B - not B, G, R. So we can reverse the band order:

In [ ]:
r[:3][::-1]

Now lets plot that:

In [ ]:
rasterio.plot.show(r[:3][::-1])

Hmmm, that's pretty dark. ```rasterio.plot``` passes the array to matplotlib, which displays it on our 8-bit monitors by multiplying each value in the array by 255. If reflectance is low - which it is, since the Earth's surface isn't a mirror - then it's going to be dark.

Again, easy fix - just specify a multiplier.

In [ ]:
rasterio.plot.show(r[:3][::-1]*5)

The warning is because having an overall multiplier means that if we have any reflectance values above 0.2, for example

In [ ]:
0.25 * 255 * 5

they will now be above 255 - which means matplotlib can't show them in an 8-bit image. So it just sets those values to the max at 255. 

There are ways of applying brightness curves to images, in order that dark values are brightened, but already light values aren't, using complex formulas dependant on the brightness value. That's a bit beyond the scope here to explain those, but if you have a complex transformation of that kind it's easy to apply - just use it in place of the ```*5``` above.

And we can do other calculations with the array as well:

In [ ]:
rasterio.plot.show(((r[3] - r[2])/(r[3] + r[2])), cmap='Greens')

That's the NDVI. Let's save that:

In [ ]:
meta.update(count=1, dtype=r.dtype, nodata=np.nan)
meta

In [ ]:
with rasterio.open('ndvi.tif', 'w', **meta) as ndvi:
    ndvi.write(((r[3] - r[2])/(r[3] + r[2])), 1)

## 5. Combining Vector and Raster Data

Let's use geopandas to read a feather file extracted from osm data. We'll add the name of the feature, and reproject to the same crs as our raster data.

In [ ]:
osm = gpd.read_feather('../sample_data/osm.feather')
osm = osm[osm['name'] == 'Port Lotniczy Poznań-Ławica im. Henryka Wieniawskiego']
osm = osm.to_crs(hls.crs)
osm

Let's make a 1km buffer around the envelope (the smallest rectangle containing all features) of the airport:

In [ ]:
buffer = osm.buffer(1000).envelope
buffer

In [ ]:
map = buffer.explore()
osm.explore(m=map, color='red')

Nice, right? We didn't do that before. That's using the library _Folium_ which produces interactive plots. I'm quite certain you've unknowingly used Folium plots before - it's commonly used for web maps.

Now, it would be nice to show this on top of our raster. We wouldn't want the whole raster though, it's too big. But we could crop it to our buffer.

<a href='https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html'>```rasterio.mask.mask()```</a> takes:
 - a raster which will be masked (opened in 'r' mode)
 - a vector shape or shapes which will be the mask layer
and some optional arguments, including
 - whether to mask or crop the raster
 - any values to use for no data

This is the tricky part about geospatial python. Not masking, I mean knowing that you can use rasterio.mask.mask()``` to mask or crop a raster. There's so much out there, and there's really no shortcuts. I suggest lots of googling "python clip raster" or the equivalent for whatever you want to do - you won't be the first to wonder. Read the StackOverflow answers, and read other people's code,  in time you'll get to know (a) the functions you use regularly, and (b) where to find the functions you don't use often. Honestly, if I've made it this far through the workshop without googling anything, it's a minor miracle.

In [ ]:
ndvi = rasterio.open('ndvi.tif')
(ndvi_crop, cropped_transform) = rasterio.mask.mask(ndvi, buffer, crop=True, nodata=np.nan)
ndvi_crop

```rasterio.mask.mask()``` returns a tuple: ```(the image, the transform)```, so we specified a tuple with names, which lets us use the names for the individual parts of the tuple.

In [ ]:
fig, ax = plt.subplots()
rasterio.plot.show(ndvi_crop, transform=cropped_transform, cmap='Greens', ax=ax)
osm.plot(ax=ax, edgecolor='red', linewidth=1, color='none')

Okay, so that's a taste of working with raster data in Python. As I said, there's a huge amout out there, and this is really just barely scratching the surface, but hopefully it's a demonstration that what can be done in GIS can be done in Python, often much more easily. And even if it takes you longer because you have to learn how to do everything, it's definitely quicker if you have to repeat the process, because next time, the code's already there and just needs to be run.

## 6. Raster Data in the Cloud

We absolutely don't have time for this now, but as we are probably all aware, processing of raster data now is massively moving into the cloud, with Google Earth Engine, Microsoft Planetary Computer, and others. Earth Engine's native interface is in javascript

![display image](https://media.giphy.com/media/tw1zMQrM2IhC8/giphy.gif)

but fear not! There is an _excellent_ python library, <a href='https://geemap.org/'>geemap</a>, with <a href='https://github.com/giswqs/earthengine-py-notebooks'>literally hundreds of example notebooks on github</a>, which, like this one, can all be run in Binder. 

![display image](https://media.giphy.com/media/s5igVaatNXUVW/giphy.gif)

## 7. Earth Observation Data Visualisation

When it comes to visualising the data, matplotlib is capable, but needs a lot of finesse to get it the way you'd want for EO, because it's simply not what it was written for. Folium is great for the interactivity, and was written for geospatial data, but it's more for web map style viewing. If you're thinking "hey, it sure would be nice if there was something written specifically for EO data", then

![display image](https://media.giphy.com/media/3zFcbgHoIXzykQc7vU/giphy.gif)

The excellent package <a href='https://github.com/raphaelquast/eomaps'>EOmaps</a> has indeed been written specifically for visualising and working with remote sensing imagery. It's highly interactive, but in a different way than Folium, allowing you to use the interactivity to help analyse your data. There are plenty of great examples on the package github repository with code you can copy - and I highly recommend it. 

# The End

Hopefully you've found this workshop useful - it's over to you now!

___
### <a href='./3_Vector_data.ipynb'>Previous</a> |